# Spliting Data
Here, we utilize the feature-selected profiles generated in the preceding module notebook [here](../0.freature_selection/), focusing on dividing the data into training, testing, and holdout sets for machine learning training.

In [24]:
import json
import pathlib
import warnings

import pandas as pd
import numpy as np

# ignoring warnings
warnings.catch_warnings(action="ignore")

catch_warnings()

## Paramters

Below are the parameters defined that are used in this notebook

In [25]:
# setting seed constants
seed = 0
np.random.seed(seed)

# directory to get all the inputs for this notebook
data_dir = pathlib.Path("../../data").resolve(strict=True)
results_dir = pathlib.Path("../../results").resolve(strict=True)
fs_dir = (results_dir / "0.feature_selection").resolve(strict=True)

# directory to store all the output of this notebook
data_split_dir = (results_dir / "1.data_splits").resolve()
data_split_dir.mkdir(exist_ok=True)

In [3]:
# data paths
fs_profile_path = (fs_dir / "cell_injury_profile_fs.csv.gz").resolve(strict=True)

# load data
fs_profile_df = pd.read_csv(fs_profile_path)

# display
print("fs profile with control: ", fs_profile_df.shape)
fs_profile_df.head()

fs profile with control:  (16701, 378)


/tmp/ipykernel_98556/727241773.py:5: DtypeWarning: Columns (0,13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  fs_profile_df = pd.read_csv(fs_profile_path)


,Compound BRD (short),Mahalanobis distance significant,Channels,Compound SMILES,Compound PubChem URL,Compound IUPAC,Compound PubChem CID,Characteristics [Cell Line],Comment [Image File Path],Well,...,Nuclei_Texture_InverseDifferenceMoment_DNA_20_0,Nuclei_Texture_InverseDifferenceMoment_DNA_5_0,Nuclei_Texture_InverseDifferenceMoment_RNA_5_0,Nuclei_Texture_SumAverage_AGP_5_0,Nuclei_Texture_SumAverage_DNA_10_0,Nuclei_Texture_SumAverage_Mito_5_0,Nuclei_Texture_SumAverage_RNA_5_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumVariance_DNA_20_0
0,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00110363/,B2,...,-0.011258,0.000098,0.057244,0.160847,-0.083034,-0.023290,-0.066369,-0.015235,-0.035909,-0.032067
1,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00110363/,B3,...,0.064689,0.025857,0.099848,0.017477,0.021300,0.058137,-0.097280,-0.073545,-0.044883,-0.015240
2,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00110363/,B4,...,0.020937,0.041060,0.119247,0.111741,0.041592,0.224199,-0.088845,0.000327,-0.003115,-0.014406
3,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00110363/,B5,...,0.006589,0.022156,0.036473,-0.013141,0.008690,0.060860,0.044924,0.040528,0.070877,0.072871
4,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00110363/,B6,...,-0.028361,0.007213,0.023068,0.110361,0.054405,0.030157,0.066480,0.038910,0.048559,0.056829


## Exploring the data set

Below is a  exploration of the selected features dataset. The aim is to identify treatments, extract metadata, and gain a understanding of the experiment's design.

Below demonstrates the amount of wells does each treatment have. 

In [4]:
# displying the amount of wells per treatments
well_treatments_counts_df = (
    fs_profile_df["Compound Name"].value_counts().to_frame().reset_index()
)

well_treatments_counts_df

,Compound Name,count
0,DMSO,9855
1,Wortmannin,600
2,Colchicine,512
3,Nocodazole,504
4,Radicicol,504
...,...,...
139,Carmustine,24
140,Thio-TEPA,24
141,Chlorambucil,24
142,Ebselen oxide,24


Below we show the amount of wells does a specific cell celluar injury has

In [5]:
# Displaying how many how wells does each cell injury have
cell_injury_well_counts = (
    fs_profile_df["injury_type"].value_counts().to_frame().reset_index()
)
cell_injury_well_counts

,injury_type,count
0,Control,9855
1,Cytoskeletal,1472
2,Miscellaneous,1302
3,Kinase,1104
4,Genotoxin,944
5,Hsp90,552
6,Redox,312
7,Saponin,288
8,HDAC,168
9,Proteasome,144


Next we wanted to extract some metadata regarding how many compound and wells are treated with a given compounds

This will be saved in the `results/0.data_splits` directory

In [6]:
meta_injury = []
for injury_type, df in fs_profile_df.groupby("injury_type"):
    # extract n_wells, n_compounds and unique compounds per injury_type
    n_wells = df.shape[0]
    unique_compounds = list(df["Compound Name"].unique())
    n_compounds = len(unique_compounds)

    # store information
    meta_injury.append([injury_type, n_wells, n_compounds, unique_compounds])

injury_meta_df = pd.DataFrame(
    meta_injury, columns=["injury_type", "n_wells", "n_compounds", "compound_list"]
).sort_values("n_wells", ascending=False)
injury_meta_df.to_csv(data_split_dir / "injury_well_counts_table.csv", index=False)

# display
print("shape:", injury_meta_df.shape)
injury_meta_df

shape: (15, 4)


,injury_type,n_wells,n_compounds,compound_list
0,Control,9855,1,[DMSO]
1,Cytoskeletal,1472,15,"[Nocodazole, Colchicine, Paclitaxel, Vinblasti..."
7,Miscellaneous,1302,39,"[L-Buthionine-(S,R)-sulfoximine, CDDO Im, Cino..."
6,Kinase,1104,13,"[Wortmannin, Staurosporine, PI-103, BEZ-235, A..."
3,Genotoxin,944,22,"[Camptothecin, CX-5461, Doxorubicin, Cladribin..."
5,Hsp90,552,3,"[Radicicol, Geldanamycin, 17-AAG]"
11,Redox,312,12,"[Menadione, PKF118-310, 4-Amino-1-naphthol (HC..."
12,Saponin,288,11,"[Digitonin, Saikosaponin A, Polygalasaponin F,..."
4,HDAC,168,5,"[AR-42, SAHA, ITF 2357, Panobinostat, Apicidin]"
8,Mitochondria,144,4,"[Antimycin A, CCCP, Rotenone, Oligomycin A]"


> Barchart showing the number of wells that are labeled with a given injury

Next, we construct the profile metadata. This provides a structured overview of how the treatments assicoated with injuries were applied, detailing the treatments administered to each plate.

This will be saved in the `results/0.data_splits` directory

In [7]:
injury_meta_dict = {}
for injury, df in fs_profile_df.groupby("injury_type"):
    # collecting treatment metadata
    plates = df["Plate"].unique().tolist()
    treatment_meta = {}
    treatment_meta["n_plates"] = len(plates)
    treatment_meta["n_wells"] = df.shape[0]
    treatment_meta["n_treatments"] = len(df["Compound Name"].unique())
    treatment_meta["associated_plates"] = plates

    # counting treatments
    treatment_counter = {}
    for treatment, df2 in df.groupby("Compound Name"):
        if treatment is np.nan:
            continue
        n_treatments = df2.shape[0]
        treatment_counter[treatment] = n_treatments

    # storing treatment counts
    treatment_meta["treatments"] = treatment_counter
    injury_meta_dict[injury] = treatment_meta

# save dictionary into a json file
with open(data_split_dir / "injury_metadata.json", mode="w") as stream:
    json.dump(injury_meta_dict, stream)

Here we build a plate metadata infromations where we look at the type of treatments and amount of wells with the treatment that are present in the dataset

This will be saved in `results/0.data_splits`

In [8]:
plate_meta = {}
for plate_id, df in fs_profile_df.groupby("Plate"):
    unique_compounds = list(df["Compound Name"].unique())
    n_treatments = len(unique_compounds)

    # counting treatments
    treatment_counter = {}
    for treatment, df2 in df.groupby("Compound Name"):
        n_treatments = df2.shape[0]
        treatment_counter[treatment] = n_treatments

    plate_meta[plate_id] = treatment_counter

# save dictionary into a json file
with open(data_split_dir / "plate_info.json", mode="w") as stream:
    json.dump(plate_meta, stream)

Set numerical labels for the treatment

In [9]:
# creating a dictionary that contains the numeric-encoded labels and write out as json file
main_labeler = {}
injury_labels_encoder = {
    name: idx for idx, name in enumerate(fs_profile_df["injury_type"].unique().tolist())
}
injury_labels_decoder = {
    idx: name for idx, name in enumerate(fs_profile_df["injury_type"].unique().tolist())
}
main_labeler["encoder"] = injury_labels_encoder
main_labeler["decoder"] = injury_labels_decoder

# write out as json file
with open(data_split_dir / "injury_codes.json", mode="w") as file_buffer:
    json.dump(main_labeler, file_buffer)

# display main_labeler
main_labeler

{'encoder': {'Control': 0,
  'Cytoskeletal': 1,
  'Hsp90': 2,
  'Kinase': 3,
  'Genotoxin': 4,
  'Miscellaneous': 5,
  'Redox': 6,
  'HDAC': 7,
  'mTOR': 8,
  'Proteasome': 9,
  'Saponin': 10,
  'Mitochondria': 11,
  'Ferroptosis': 12,
  'Tannin': 13,
  'Nonspecific reactive': 14},
 'decoder': {0: 'Control',
  1: 'Cytoskeletal',
  2: 'Hsp90',
  3: 'Kinase',
  4: 'Genotoxin',
  5: 'Miscellaneous',
  6: 'Redox',
  7: 'HDAC',
  8: 'mTOR',
  9: 'Proteasome',
  10: 'Saponin',
  11: 'Mitochondria',
  12: 'Ferroptosis',
  13: 'Tannin',
  14: 'Nonspecific reactive'}}

Now that we have assigned numerical labels to each type of cell injury, we can replace the corresponding injury names with these numerical values to meet the requirements of machine learning algorithms.

In [10]:
# updating main dataframe with numerical labels that represents cell injury
# this will be saved as an "injury_code"
injury_code = fs_profile_df["injury_type"].apply(
    lambda injury: injury_labels_encoder[injury]
)

# add the injury code into the main data set
fs_profile_df.insert(0, "injury_code", injury_code)

# # display new injury column
print(fs_profile_df["injury_type"].unique())
print(fs_profile_df["injury_code"].unique())

['Control' 'Cytoskeletal' 'Hsp90' 'Kinase' 'Genotoxin' 'Miscellaneous'
 'Redox' 'HDAC' 'mTOR' 'Proteasome' 'Saponin' 'Mitochondria' 'Ferroptosis'
 'Tannin' 'Nonspecific reactive']
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]


## Data Splitting 
---

### Holdout Dataset

Here we collected out holdout dataset. The holdout dataset is a subset of the dataset that is not used during model training or tuning. Instead, it is reserved solely for evaluating the model's performance after it has been trained.

In this notebook, we will include three different types of held-out datasets before proceeding with our machine learning training and evaluation.
 - Plate hold out 
 - treatment hold out 
 - well hold out 

Each of these held outdata will be stored in the `results/1.data_splits` directory 


### Plate Holdout

Plates are randomly selected based on their Plate ID and save them as our `plate_holdout` data.

In [11]:
# plate
seed = 0
n_plates = 10

# setting random seed globally
np.random.seed(seed)

# selecting plates randomly from a list
selected_plates = (
    np.random.choice(fs_profile_df["Plate"].unique().tolist(), (n_plates, 1))
    .flatten()
    .tolist()
)
plate_holdout_df = fs_profile_df.loc[fs_profile_df["Plate"].isin(selected_plates)]

# take the indices of the held out data frame and use it to drop those samples from
# the main dataset. And then check if those indices are dropped
plate_idx_to_drop = plate_holdout_df.index.tolist()
fs_profile_df = fs_profile_df.drop(plate_idx_to_drop)
assert all(
    [
        True if num not in fs_profile_df.index.tolist() else False
        for num in plate_idx_to_drop
    ]
), "index to be dropped found in the main dataframe"

# saving the holdout data
plate_holdout_df.to_csv(
    data_split_dir / "plate_holdout.csv.gz", index=False, compression="gzip"
)

# display
print("plate holdout shape:", plate_holdout_df.shape)
plate_holdout_df.head()

plate holdout shape: (1948, 379)


,injury_code,Compound BRD (short),Mahalanobis distance significant,Channels,Compound SMILES,Compound PubChem URL,Compound IUPAC,Compound PubChem CID,Characteristics [Cell Line],Comment [Image File Path],...,Nuclei_Texture_InverseDifferenceMoment_DNA_20_0,Nuclei_Texture_InverseDifferenceMoment_DNA_5_0,Nuclei_Texture_InverseDifferenceMoment_RNA_5_0,Nuclei_Texture_SumAverage_AGP_5_0,Nuclei_Texture_SumAverage_DNA_10_0,Nuclei_Texture_SumAverage_Mito_5_0,Nuclei_Texture_SumAverage_RNA_5_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumVariance_DNA_20_0
1044,0,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00110368/,...,-0.062886,0.011554,0.087596,0.163291,0.058129,-0.007010,0.100495,0.093309,0.108031,0.139935
1045,0,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00110368/,...,-0.018239,0.059690,0.118740,0.031366,-0.001880,0.124516,-0.115299,0.065540,0.095688,0.097536
1046,0,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00110368/,...,-0.054246,0.013374,0.010342,0.070002,0.002531,0.079322,0.127617,0.071349,0.025576,0.051150
1047,0,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00110368/,...,-0.015467,0.016590,0.077109,0.007670,0.039326,0.022608,0.012423,0.076461,0.076174,0.098298
1048,0,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00110368/,...,-0.006360,0.040196,0.104461,0.081361,0.013528,0.012501,-0.044112,0.043685,0.063887,0.066430


### Treatment holdout

To establish our treatment holdout, we first need to find the number of treatments and wells associated with a specific cell injury, considering the removal of randomly selected plates from the previous step.

To determine which cell injuries should be considered for a single treatment holdout, we establish a threshold of 10 unique compounds. This means that a cell injury type must have at least 10 unique compounds to qualify for selection in the treatment holdout. Any cell injury types failing to meet this criterion will be disregarded.

Once the cell injuries are identified for treatment holdout, we select our holdout treatment by grouping each injury type and choosing the treatment with the fewest wells. This becomes our treatment holdout dataset.

In [23]:
injury_treatment_metadata = (
    fs_profile_df.groupby(["injury_type", "Compound Name"])
    .size()
    .reset_index(name="n_wells")
)
injury_treatment_metadata

,injury_type,Compound Name,n_wells
0,Control,DMSO,8783
1,Cytoskeletal,ARQ 621,12
2,Cytoskeletal,Citreoviridin,18
3,Cytoskeletal,Citrinin,18
4,Cytoskeletal,Colchicine,457
...,...,...,...
139,Tannin,Corilagin,18
140,Tannin,Gallotannin,24
141,Tannin,Punicalagin,18
142,mTOR,Rapamycin,42


In [27]:
# setting random seed
min_treatments_per_injury = 10

# Filter out the injury types for which we can select a complete treatment.
# We are using a threshold of 10. If an injury type is associated with fewer than 10 compounds,
# we do not conduct treatment holdout on those injury types.
accepted_injuries = []
for injury_type, df in injury_treatment_metadata.groupby("injury_type"):
    n_treatments = df.shape[0]
    if n_treatments >= min_treatments_per_injury:
        accepted_injuries.append(df)

accepted_injuries = pd.concat(accepted_injuries)

# Next, we select the treatment that will be held out within each injury type.
# We group treatments based on injury type and choose the treatment with the fewest wells
# as our holdout.
selected_treatments_to_holdout = []
for injury_type, df in accepted_injuries.groupby("injury_type"):
    held_treatment = df.min().iloc[1]
    selected_treatments_to_holdout.append([injury_type, held_treatment])

# convert to dataframe
selected_treatments_to_holdout = pd.DataFrame(
    selected_treatments_to_holdout, columns="injury_type held_treatment".split()
)

print("Below are the accepted cell injuries and treatments to be held out")
selected_treatments_to_holdout

Below are the accepted cell injuries and treatments to be held out


,injury_type,held_treatment
0,Cytoskeletal,ARQ 621
1,Genotoxin,Aphidicolin
2,Kinase,AZD 1152-HQPA
3,Miscellaneous,Aloisine RP106
4,Redox,4-Amino-1-naphthol (HCl)
5,Saponin,Bacopasaponin C


In [28]:
# select all wells that have the treatments to be heldout
treatment_holdout_df = fs_profile_df.loc[
    fs_profile_df["Compound Name"].isin(
        selected_treatments_to_holdout["held_treatment"]
    )
]

# take the indices of the held out data frame and use it to drop those samples from
# the main dataset. And then check if those indices are dropped
treatment_idx_to_drop = treatment_holdout_df.index.tolist()
fs_profile_df = fs_profile_df.drop(treatment_idx_to_drop)
assert all(
    [
        True if num not in fs_profile_df.index.tolist() else False
        for num in treatment_idx_to_drop
    ]
), "index to be dropped found in the main dataframe"
# saving the holdout data
treatment_holdout_df.to_csv(
    data_split_dir / "treatment_holdout.csv.gz", index=False, compression="gzip"
)

# display
print("Treatment holdout shape:", treatment_holdout_df.shape)
treatment_holdout_df.head()

Treatment holdout shape: (126, 379)


,injury_code,Compound BRD (short),Mahalanobis distance significant,Channels,Compound SMILES,Compound PubChem URL,Compound IUPAC,Compound PubChem CID,Characteristics [Cell Line],Comment [Image File Path],...,Nuclei_Texture_InverseDifferenceMoment_DNA_20_0,Nuclei_Texture_InverseDifferenceMoment_DNA_5_0,Nuclei_Texture_InverseDifferenceMoment_RNA_5_0,Nuclei_Texture_SumAverage_AGP_5_0,Nuclei_Texture_SumAverage_DNA_10_0,Nuclei_Texture_SumAverage_Mito_5_0,Nuclei_Texture_SumAverage_RNA_5_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumVariance_DNA_20_0
10865,1,BRD-K43436647,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",C#CC[C@H](C1=NC2=C(C=CC(=C2)Cl)C(=O)N1NC3=CC=C...,https://pubchem.ncbi.nlm.nih.gov/compound/2511...,N-(3-aminopropyl)-N-[(1R)-1-(3-anilino-7-chlor...,25110841.0,U2OS,/incoming/BR00114093/,...,-0.718839,0.049142,-1.740741,0.253751,-2.058566,2.938515,1.600429,0.843067,0.216721,0.809478
10866,1,BRD-K43436647,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",C#CC[C@H](C1=NC2=C(C=CC(=C2)Cl)C(=O)N1NC3=CC=C...,https://pubchem.ncbi.nlm.nih.gov/compound/2511...,N-(3-aminopropyl)-N-[(1R)-1-(3-anilino-7-chlor...,25110841.0,U2OS,/incoming/BR00114093/,...,0.014927,0.100368,0.080819,0.173918,-0.397879,0.155645,0.373518,0.133463,0.120955,0.235244
10867,1,BRD-K43436647,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",C#CC[C@H](C1=NC2=C(C=CC(=C2)Cl)C(=O)N1NC3=CC=C...,https://pubchem.ncbi.nlm.nih.gov/compound/2511...,N-(3-aminopropyl)-N-[(1R)-1-(3-anilino-7-chlor...,25110841.0,U2OS,/incoming/BR00114093/,...,-0.051267,-0.009367,0.037545,-0.041294,-0.375493,0.098880,0.095514,-0.027942,-0.001075,0.017841
10868,1,BRD-K43436647,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",C#CC[C@H](C1=NC2=C(C=CC(=C2)Cl)C(=O)N1NC3=CC=C...,https://pubchem.ncbi.nlm.nih.gov/compound/2511...,N-(3-aminopropyl)-N-[(1R)-1-(3-anilino-7-chlor...,25110841.0,U2OS,/incoming/BR00114093/,...,-0.015987,0.000596,-0.035705,0.055867,-0.018554,-0.020018,0.131029,0.050271,0.022755,0.033448
10869,1,BRD-K43436647,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",C#CC[C@H](C1=NC2=C(C=CC(=C2)Cl)C(=O)N1NC3=CC=C...,https://pubchem.ncbi.nlm.nih.gov/compound/2511...,N-(3-aminopropyl)-N-[(1R)-1-(3-anilino-7-chlor...,25110841.0,U2OS,/incoming/BR00114093/,...,0.017609,0.066929,0.083370,-0.186292,-0.096957,-0.063968,-0.073064,0.061575,0.050377,0.049060


### Well holdout 

To generate the well hold out data, each plate was iterated and random wells were selected. However, an additional step was condcuting which was to seperate the control wells and the treated wells, due to the large label imbalance with the controls. Therefore, 5 wells were randomly selected and 10 wells were randomly selected from each individual plate

In [15]:
# parameters
seed = 0
n_controls = 5
n_samples = 10

# setting random seed globally
np.random.seed(seed)

# collecting randomly select wells based on treatment
wells_heldout_df = []
for treatment, df in fs_profile_df.groupby("Plate", as_index=False):
    # separate control wells and rest of all wells since there is a huge label imbalance
    # selected 5 control wells and 10 random wells from the plate
    df_control = df.loc[df["Compound Name"] == "DMSO"].sample(
        n=n_controls, random_state=seed
    )
    df_treated = df.loc[df["Compound Name"] != "DMSO"].sample(
        n=n_samples, random_state=seed
    )

    # concatenate those together
    well_heldout = pd.concat([df_control, df_treated])

    wells_heldout_df.append(well_heldout)

# genearte treatment holdout dataframe
wells_heldout_df = pd.concat(wells_heldout_df)

# take the indices of the held out data frame and use it to drop those samples from
# the main dataset. And then check if those indices are dropped
wells_idx_to_drop = wells_heldout_df.index.tolist()
fs_profile_df = fs_profile_df.drop(wells_idx_to_drop)
assert all(
    [
        True if num not in fs_profile_df.index.tolist() else False
        for num in treatment_idx_to_drop
    ]
), "index to be dropped found in the main dataframe"

# saving the holdout data
wells_heldout_df.to_csv(
    data_split_dir / "wells_holdout.csv.gz", index=False, compression="gzip"
)

# display
print("Wells holdout shape:", wells_heldout_df.shape)
wells_heldout_df.head()

Wells holdout shape: (1125, 379)


,injury_code,Compound BRD (short),Mahalanobis distance significant,Channels,Compound SMILES,Compound PubChem URL,Compound IUPAC,Compound PubChem CID,Characteristics [Cell Line],Comment [Image File Path],...,Nuclei_Texture_InverseDifferenceMoment_DNA_20_0,Nuclei_Texture_InverseDifferenceMoment_DNA_5_0,Nuclei_Texture_InverseDifferenceMoment_RNA_5_0,Nuclei_Texture_SumAverage_AGP_5_0,Nuclei_Texture_SumAverage_DNA_10_0,Nuclei_Texture_SumAverage_Mito_5_0,Nuclei_Texture_SumAverage_RNA_5_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumVariance_DNA_20_0
4994,0,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00109990/,...,-0.033483,-0.030215,-0.000435,-0.079334,0.020319,0.165035,-0.031037,0.025980,-0.000591,0.000401
5058,0,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00109990/,...,0.020964,-0.018817,0.024526,-0.005947,0.038915,0.069078,-0.073076,-0.027233,-0.000291,0.014113
5050,0,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00109990/,...,0.094199,0.064575,0.161873,-0.041443,-0.063390,0.193849,-0.124109,0.034587,0.073446,0.099643
5035,0,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00109990/,...,0.034564,-0.037050,-0.037351,0.060268,0.012708,0.061491,-0.040638,-0.093101,-0.116318,-0.111381
4991,0,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00109990/,...,0.037054,0.029132,-0.070626,0.132449,-0.002434,-0.023357,0.016580,-0.085577,-0.061243,-0.080604


## Saving training dataset

Once the data holdout has been generated, the next step is to save the training dataset that will serve as the basis for training the multi-class logistic regression model.

In [16]:
# Showing the amount of data we have after removing the holdout data
meta_injury = []
for injury_type, df in fs_profile_df.groupby("injury_type"):
    # extract n_wells, n_compounds and unique compounds per injury_type
    n_wells = df.shape[0]
    injury_code = df["injury_code"].unique()[0]
    unique_compounds = list(df["Compound Name"].unique())
    n_compounds = len(unique_compounds)

    # store information
    meta_injury.append(
        [injury_type, injury_code, n_wells, n_compounds, unique_compounds]
    )

# creating data frame
injury_meta_df = pd.DataFrame(
    meta_injury,
    columns=["injury_type", "injury_code", "n_wells", "n_compounds", "compound_list"],
).sort_values("n_wells", ascending=False)
injury_meta_df.to_csv(data_split_dir / "injury_well_counts_table.csv", index=False)

# display
injury_meta_df

,injury_type,injury_code,n_wells,n_compounds,compound_list
0,Control,0,8408,1,[DMSO]
1,Cytoskeletal,1,1102,14,"[Nocodazole, Colchicine, Paclitaxel, Vinblasti..."
7,Miscellaneous,5,1006,38,"[L-Buthionine-(S,R)-sulfoximine, CDDO Im, Cino..."
6,Kinase,3,750,12,"[Wortmannin, Staurosporine, PI-103, BEZ-235, S..."
3,Genotoxin,4,738,21,"[Camptothecin, CX-5461, Doxorubicin, Cladribin..."
5,Hsp90,2,418,3,"[Radicicol, Geldanamycin, 17-AAG]"
11,Redox,6,215,11,"[Menadione, PKF118-310, Dunnione, MGR2, SIN-1 ..."
12,Saponin,10,163,10,"[Digitonin, Saikosaponin A, Polygalasaponin F,..."
4,HDAC,7,138,5,"[AR-42, SAHA, ITF 2357, Panobinostat, Apicidin]"
10,Proteasome,9,117,4,"[Carfilzomib, Bortezomib, (S)-MG132, (R)-MG132]"


In [17]:
# shape of the update training and testing dataset after removing holdout
print("training shape after removing holdouts", fs_profile_df.shape)
fs_profile_df.head()

training shape after removing holdouts (13502, 379)


,injury_code,Compound BRD (short),Mahalanobis distance significant,Channels,Compound SMILES,Compound PubChem URL,Compound IUPAC,Compound PubChem CID,Characteristics [Cell Line],Comment [Image File Path],...,Nuclei_Texture_InverseDifferenceMoment_DNA_20_0,Nuclei_Texture_InverseDifferenceMoment_DNA_5_0,Nuclei_Texture_InverseDifferenceMoment_RNA_5_0,Nuclei_Texture_SumAverage_AGP_5_0,Nuclei_Texture_SumAverage_DNA_10_0,Nuclei_Texture_SumAverage_Mito_5_0,Nuclei_Texture_SumAverage_RNA_5_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumVariance_DNA_20_0
0,0,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00110363/,...,-0.011258,0.000098,0.057244,0.160847,-0.083034,-0.023290,-0.066369,-0.015235,-0.035909,-0.032067
1,0,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00110363/,...,0.064689,0.025857,0.099848,0.017477,0.021300,0.058137,-0.097280,-0.073545,-0.044883,-0.015240
2,0,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00110363/,...,0.020937,0.041060,0.119247,0.111741,0.041592,0.224199,-0.088845,0.000327,-0.003115,-0.014406
3,0,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00110363/,...,0.006589,0.022156,0.036473,-0.013141,0.008690,0.060860,0.044924,0.040528,0.070877,0.072871
4,0,NaN,No,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",CS(=O)C,https://pubchem.ncbi.nlm.nih.gov/compound/679,methylsulfinylmethane,679.0,U2OS,/incoming/BR00110363/,...,-0.028361,0.007213,0.023068,0.110361,0.054405,0.030157,0.066480,0.038910,0.048559,0.056829


In [18]:
fs_profile_df.to_csv(
    data_split_dir / "training_data.csv.gz", index=False, compression="gzip"
)